# Anonymization of Personal Details

The goal of this project was to use matrices and the principles of linear algebra to anonymize personal details. This project is largely theoretical and consisted of two parts. The first part is a proof of the principle that matrix multiplication can be used to anonymize details without affecting the quality of a machine learning model. The second part is a brief implementation of this principle on a dataset to compare model accuracy results using the transformed and original data.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Импорты" data-toc-modified-id="Импорты-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Импорты</a></span></li></ul></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

### Импорты

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2


## Загрузка данных

In [2]:
try:
    insurance = pd.read_csv('datasets/insurance.csv')

except:
    insurance = pd.read_csv('/datasets/insurance.csv')

In [3]:
display(insurance.head())
insurance.info()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
features_df = insurance.drop(columns='Страховые выплаты')
target_df = insurance['Страховые выплаты']


## Умножение матриц

Пример: формула линейной регрессии

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

$$
a = Xw
$$

$$
X_m = XP
$$

$$
a_m = X_mw_m
$$

:::::

**Вопрос:
Равны ли *a* и *a<sub>m</sub>* ?**





$$
a = Xw
$$
$$
w = (X^TX)^{-1}X^Ty
$$
$$
X_m = XP
$$

___
$$
w_m = (X_m^TX_m)^{-1}X_m^Ty
$$

$$
w_m = ((XP)^TXP)^{-1}(XP)^Ty
$$

$$
w_m = (P^TX^TXP)^{-1}P^TX^Ty
$$

$$
w_m = (P^T(X^TX)P)^{-1}P^TX^Ty
$$

$$
w_m = (P^{-1}(X^TX)^{-1}(P^T)^{-1})P^TX^Ty
$$

$$
w_m = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

$$
w_m = P^{-1}(X^TX)^{-1}EX^Ty
$$

$$
w_m = P^{-1}(X^TX)^{-1}X^Ty
$$

$$
w_m = P^{-1}w
$$

___

$$
a_m = X_mw_m
$$

$$
a_m = XP(P^{-1}w)
$$

$$
a_m = XPP^{-1}w
$$

$$
a_m = XEw
$$

$$
a_m = Xw
$$

$$
a_m = a
$$


Качество линейной регрессии не изменится при умножении признаков на обратимую матрицу. Это означает, что можно использовать обратимые матрицы чтобы зашифровать признаки таблиц.

**Обоснование:**
Выше я показал аналитически, что предсказания не изменятся при умножении признаков на обратимую матирцу, поэтому качество модели будет одинаково.

*Пример обратной матрицы*

Тут и позже в проекте заметно, что есть ненулевые значения, которые теоритически должны быть равны нулью. Это обясняется работой питона и не является проблемой, поскольку все эти значения крайно маленькие (e-16 – e-18): можно считать их нулем. 

Есть аналогичная ситуация в других случаях проекта, когда значение крайне близко, но не равно тому значению, которым оно должно быть. Например, так происходит при разшифровнии матрицы после её переобразования с помощью обратимой матрицы – пример этого тоже находится ниже.

Можно использовать `numpy.allclose()` чтобы проверять подобные случаи.

In [5]:
random_matrix = np.random.randint(10, size=(4,4))
random_matrix_inv = np.linalg.inv(random_matrix)
print(random_matrix@random_matrix_inv)

[[ 1.00000000e+00 -5.55111512e-17  0.00000000e+00 -1.11022302e-16]
 [ 2.77555756e-17  1.00000000e+00  1.11022302e-16  0.00000000e+00]
 [-8.32667268e-17  1.11022302e-16  1.00000000e+00  1.66533454e-16]
 [ 1.11022302e-16 -5.55111512e-17  0.00000000e+00  1.00000000e+00]]


def generate_invertible_matrix(size):
    try:
        matrix = np.random.normal(size=(size, size))
        # проверим матрицу на обратимость, если нет, пробуем сгенерировать еще раз
        # таким образом гарантируем, что матрица стопроцентно будет обратимой
        np.linalg.inv(matrix)
    except np.linalg.LinAlgError:
        matrix = generate_invertible_matrix()
    
    return matrix

In [6]:
crypt_features = features_df@random_matrix
uncrypt_features = crypt_features@random_matrix_inv
display(features_df[:10])
print()
display(crypt_features[:10])
print()
display(uncrypt_features[:10])
print()
display(np.allclose(features_df, uncrypt_features))

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
5,1,43.0,41000.0,2
6,1,39.0,39700.0,2
7,1,25.0,38600.0,4
8,1,36.0,49700.0,1
9,1,32.0,51700.0,1


,0,1,2,3
0,99298.0,198449.0,99369.0,446578.0
1,76099.0,152053.0,76186.0,342193.0
2,42058.0,84029.0,42116.0,189116.0
3,83456.0,166835.0,83488.0,375402.0
4,52265.0,104429.0,52315.0,235017.0
5,82109.0,164058.0,82179.0,369195.0
6,79501.0,158854.0,79563.0,357479.0
7,77287.0,154454.0,77311.0,347541.0
8,99488.0,198844.0,99549.0,447458.0
9,103480.0,206840.0,103533.0,465442.0


,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-2.273737e-12,46.0,38000.0,1.000000e+00
2,-1.818989e-12,29.0,21000.0,5.002221e-12
3,-6.366463e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,5.456968e-12
5,1.000000e+00,43.0,41000.0,2.000000e+00
6,1.000000e+00,39.0,39700.0,2.000000e+00
7,1.000000e+00,25.0,38600.0,4.000000e+00
8,1.000000e+00,36.0,49700.0,1.000000e+00
9,1.000000e+00,32.0,51700.0,1.000000e+00


True

## Алгоритм преобразования

**Алгоритм**

Я использовал обратные матрицы чтобы преобразовать данные. Я решил создать новый класс и функции для этого потому, что тогда обратимая матрица создана при каждого вызова класса, позвольяющее одновременно создавать несколько разных моделей с разными матрицами.

In [7]:
class feat_crypt :
    def encrypt(self, features_dataframe):
        self.lock = np.random.randint(10, size=(features_dataframe.shape[1], features_dataframe.shape[1]))
        self.key = np.linalg.inv(self.lock)
        crypted = features_dataframe@self.lock
        return crypted
    def uncrypt(self, crypted_features):
        uncrypted = crypted_features@self.key
        return uncrypted

**Обоснование**

Показано выше, что такой подход достоверен, но также ниже есть пример того, что можно таким образом зашифровать и разшифровать данные без изменений в этих данных.

In [8]:
crypt = feat_crypt()
crypt_test = crypt.encrypt(features_df)
uncrypt_test = crypt.uncrypt(crypt_test)
display(features_df[10:20], crypt_test[10:20], uncrypt_test[10:20], 
        f'Одинаковы ли оригианальная матрица с признаками и матрица, полученная после шифрования и разшифрования: {np.allclose(features_df, uncrypt_test)}')

,Пол,Возраст,Зарплата,Члены семьи
10,1,25.0,36600.0,1
11,1,38.0,29300.0,0
12,0,23.0,39500.0,3
13,0,21.0,55000.0,0
14,0,40.0,43700.0,1
15,1,34.0,23300.0,0
16,1,26.0,48900.0,2
17,1,41.0,33200.0,2
18,1,42.0,49700.0,0
19,1,27.0,36900.0,0


,0,1,2,3
10,110013.0,256413.0,73381.0,219660.0
11,88211.0,205408.0,58871.0,175881.0
12,118702.0,276711.0,79164.0,237061.0
13,165168.0,385168.0,110147.0,330042.0
14,131426.0,306229.0,87681.0,262285.0
15,70179.0,163376.0,46843.0,139873.0
16,146927.0,342530.0,97989.0,293467.0
17,99947.0,232750.0,66694.0,199297.0
18,149443.0,348240.0,99699.0,298289.0
19,110923.0,258520.0,73994.0,221459.0


,0,1,2,3
10,1.000000e+00,25.0,36600.0,1.000000e+00
11,1.000000e+00,38.0,29300.0,4.547474e-12
12,0.000000e+00,23.0,39500.0,3.000000e+00
13,0.000000e+00,21.0,55000.0,3.637979e-12
14,7.275958e-12,40.0,43700.0,1.000000e+00
15,1.000000e+00,34.0,23300.0,1.364242e-12
16,1.000000e+00,26.0,48900.0,2.000000e+00
17,1.000000e+00,41.0,33200.0,2.000000e+00
18,1.000000e+00,42.0,49700.0,6.366463e-12
19,1.000000e+00,27.0,36900.0,3.637979e-12


'Одинаковы ли оригианальная матрица с признаками и матрица, полученная после шифрования и разшифрования: True'

## Проверка алгоритма

In [9]:
class linreg :
    def fit(self, features_train, target_train):
        Z = np.concatenate((np.ones((features_train.shape[0], 1)), features_train), axis=1)
        w = (np.linalg.inv(Z.T.dot(Z))).dot(Z.T).dot(target_train)
        self.w = w[1:]
        self.w0 = w[0]
    def predict(self, features_test):
        a = features_test.dot(self.w) + self.w0
        return a
    

In [10]:
model = linreg()
model.fit(features_df, target_df)
predictions = model.predict(features_df)
display(pd.DataFrame({'predictions':predictions, 'target':target_df}))
print(f'R2 score: {r2(target_df, predictions)}')

,predictions,target
0,0.511727,0
1,0.684316,1
2,0.093734,0
3,-0.222589,0
4,0.065084,0
...,...,...
4995,0.028390,0
4996,0.253367,0
4997,-0.256970,0
4998,-0.190992,0


R2 score: 0.4249455028666801


In [11]:
r2_scores = []
no_crypt_r2 = r2(target_df, predictions)
for i in range(7):
    crypt_func = feat_crypt()
    crypt_df = crypt_func.encrypt(features_df)
    lin_model = linreg()
    model.fit(crypt_df, target_df)
    pred = model.predict(crypt_df)
    r2_scores.append(r2(target_df, pred))
    

display(pd.Series(r2_scores, name='R2 scores from encrypted predictions').to_frame(),
        pd.Series(no_crypt_r2, name='R2 score of unencrypted predictions').to_frame())
print(r2_scores, '\n', no_crypt_r2)

,R2 scores from encrypted predictions
0,0.424946
1,0.424946
2,0.424946
3,0.424946
4,0.424946
5,0.424946
6,0.424946


,R2 score of unencrypted predictions
0,0.424946


[0.42494550285076804, 0.424945502866353, 0.4249455027606317, 0.42494550286628097, 0.4249455028666804, 0.4249455028665482, 0.42494550286667965] 
 0.4249455028666801


Как показано в предыдушей ячейке, алгоритм преобразование не влияет на результаты или качество линейной регрессии. Есть очень маленькие разницы в R2 score, но их можно считать незначительными. Эти разницы присутствуют по тем причинам, как раньше с неточностью обратных матриц – они не связаны с математикой или теорией, а с тем, как питон работает и вычисляет их.